In [1]:
!pip install opencv-python

In [2]:
import sys
import math
import cv2 as cv
import numpy as np
import os


In [14]:
def Preprocess(File_Path):
    default_file = File_Path
    filename = default_file

    # Loads an image
    src = cv.imread(default_file, cv.IMREAD_GRAYSCALE)

    #cv.imshow("Raw Image", src)
    #print("Raw Image Size",np.shape(src))

    # Check if image is loaded fine
    if src is None:
        print('Error opening image!')
        print('Usage: hough_lines.py [image_name -- default ' + default_file + '] \n')
        return -1

    xscale = 0.4
    
    yscale = 1
    
    src = cv.resize(src, None, fx=xscale, fy=yscale)

    #cv.imshow("Resized Image", src)
    #print("Resized Image Size",np.shape(src))
    
    src = src[int(1450*yscale):int(2100*yscale), int(800*xscale):int(2500*xscale)]

    cv.imshow("AOI Image", src)
    #print("AOI Image Size",np.shape(src))

    
    dst = cv.Canny(src, 150, 500, 80, 3)

    # Copy edges to the images that will display the results in BGR
    cdstP = cv.cvtColor(src, cv.COLOR_GRAY2BGR)
    

    linesP = cv.HoughLinesP(dst, 1, np.pi / 180, 20, None, 20, 5)

    if linesP is not None:
        for i in range(0, len(linesP)):
            l = linesP[i][0]
            cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0, 0, 255), 1, cv.LINE_AA)

    cv.imshow(filename, dst)
    
    cv.imshow("Hough_P_Image", cdstP)
    
    cv.waitKey()
    
    if linesP is not None:
        linesP = sorted(linesP, key=lambda x: np.sqrt((x[0][2] - x[0][0])**2 + (x[0][3] - x[0][1])**2), reverse=True)[:100]
    else:
        linesP = []  # If no lines are detected, initialize an empty list

    return dst, linesP


In [ ]:
img, lines = Preprocess(r"C:\Users\bchko\Downloads\CEE4990\images\Vernier_Scale_1_05.jpg")
print(np.shape(img))

cv.imshow("Vernier_Scale_T_20.jpg", img)
cv.waitKey()


In [5]:
# Directory containing images
image_directory = r"C:\Users\bchko\Downloads\CEE4990\images"

# List to store processed data
dataset = np.empty(shape=(0,), dtype=[('image', object), ('label', '<U2')])

# Initialize lists to store images and labels separately
x1 = []
x2 = []
y = []

# Iterate over each image file in the directory
for filename in os.listdir(image_directory):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Extract the last two digits of the filename before the ".jpg" extension
        label = filename[-6:-4]  # Assuming the filename format is XXXXXYY.jpg
        
        # Construct full path to the image file
        image_path = os.path.join(image_directory, filename)
        
        # Preprocess the image
        img, lines = Preprocess(image_path)
        
        # Append processed data to the dataset along with the label
        dataset = np.append(dataset, np.array([(img, int(label))], dtype=dataset.dtype))

        lines = np.squeeze(lines)

        # Get the indices that would sort the array along the first axis based on the first value in each line
        sorted_indices = np.argsort(lines[:, 0])
        
        # Reorder the lines of the array based on the sorted indices
        Reordered_Lines = lines[sorted_indices]

        Reordered_Lines[:, [0, 2]] -= Reordered_Lines[0,0]

        #print(np.shape(Reordered_Lines))
            
        # Append image and label to x and y lists
        #print(np.shape(lines))
        x2.append(Reordered_Lines)
        x1.append(img)
        y.append(int(label))



# Convert x and y lists to NumPy arrays
x1 = np.array(x1)
x2 = np.array(x2)
y = np.array(y)

# Now x contains the array of images, and y contains the array of labels

for count, value in enumerate(y):
    if np.shape(x2[count]) == (100,4):
        print('good')
    else:
        print(value)

good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good
good


In [6]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets with an 80-20 split
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y, test_size=0.2)
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y, test_size=0.2)



In [7]:
print (x_train1.shape) 
print (x_train2.shape) 
print (x_test1.shape) 
print (x_test2.shape)

(23, 650, 680)
(23, 100, 4)
(6, 650, 680)
(6, 100, 4)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense

# Define the image dimensions
img_rows, img_cols = 650, 680  # Example image size, adjust as necessary

# Create the regression model
model_img = Sequential([
    Input(shape=(img_rows, img_cols, 1)),  # Assuming grayscale images, hence '1' at the end for channels
    Conv2D(50, kernel_size=3, activation='relu'),
    Conv2D(50, kernel_size=3, activation='relu'),
    Conv2D(40, kernel_size=3, activation='relu'),
    Conv2D(20, kernel_size=3, activation='relu'),
    Conv2D(20, kernel_size=3, activation='relu'),
    Flatten(),
    Dense(100, activation='relu'),  # Increase the number of units for better representation
    Dense(100, activation='relu'), 
    Dense(100, activation='relu'),
    Dense(1)  # Output layer for regression, with a single output neuron
])

In [9]:
# Compile the model
model_img.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Print model summary
model_img.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 648, 678, 50)   │           500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 646, 676, 50)   │        22,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 644, 674, 40)   │        18,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 642, 672, 20)   │         7,220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 640, 670, 20)   │         3,620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8576000)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │   857,600,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 857,672,331 (3.20 GB)

 Trainable params: 857,672,331 (3.20 GB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model_img.fit (x_train1, y_train1, epochs = 10, validation_data=(x_test1, y_test1), verbose = 1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 56s 56s/step - loss: 96.0149 - mae: 7.6032 - val_loss: 1046344960.0000 - val_mae: 32346.6191
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 198s 198s/step - loss: 1048997440.0000 - mae: 32381.4316 - val_loss: 12361552.0000 - val_mae: 3478.0432
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 200s 200s/step - loss: 11744608.0000 - mae: 3400.2312 - val_loss: 465110.2500 - val_mae: 532.3168
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 242s 242s/step - loss: 387480.3125 - mae: 501.3701 - val_loss: 147935.6406 - val_mae: 313.3823
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 188s 188s/step - loss: 159608.9375 - mae: 342.9713 - val_loss: 404260.7500 - val_mae: 609.5380
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 268s 268s/step - loss: 423903.6562 - mae: 631.7726 - val_loss: 641655.6250 - val_mae: 786.3173
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 242s 242s/step - loss: 654745.3750 - mae: 798.6270 - val_loss: 712726.9375 - val_mae: 829.9392
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 181s 181s/step - loss: 715382.5625 - 

In [ ]:

# Define the model
model_lines = Sequential([
    Flatten(input_shape=(100, 4)),  # Flatten the input to a 1D array
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(100, activation='relu'),
    Dense(50, activation='relu'),
    Dense(1)  # Output layer for regression with a single output neuron
])

In [ ]:
# Compile the model
model_lines.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


model_lines.fit (x_train2, y_train2)

# Print model summary
model_lines.summary()